In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from operator import add
#import seaborn as sns

In [17]:
pool=np.loadtxt("./feasible_data_myringwip.csv", delimiter=",",skiprows=0, dtype=np.float32)
train,test=data_split_size(pool,2400)
np.savetxt("./train_2400.txt", train ,delimiter=" ")
np.savetxt("./test_2400.txt", test ,delimiter=" ")

In [12]:
def data_split_size(data,size):
      #on a given dataset return the splitted data=> train_data(based on size),validate_data(leftover)
      a_list=np.arange(data.shape[0])
      np.random.shuffle(a_list)
      alist=a_list[0:size]
      train_data=data[alist]
      d=np.arange(data.shape[0])
      leftover=np.delete(d,alist)
      validate_data=data[leftover]
      return train_data,validate_data

def calc_residual(truth,pred):
    delz= np.subtract(truth,pred)
    residual=np.divide(np.abs(delz),np.abs(truth))
    sum_residual= np.sum(residual)/truth.shape[0]
    print('residual is:',residual.shape,'sum residual is:',sum_residual)

    
def calc_deviation(truth,pred):
    delz= np.subtract(truth,pred)
    var= np.sum(np.square(delz))/truth.shape[0]
    print('variance is:',var)
    std= np.power(var,0.5)
    print('std is:',std)
    return std
    
def label_data(data,stest_pred):
      # create label for data(if predicted vale is >/< 5% of error then it labels it '1' or else it is '0')
      ones=np.ones(stest_pred.shape[0])
      zeros= np.zeros(stest_pred.shape[0])
      #print('test shape:',test_data[:,-1].shape,'zeros shape:',zeros.shape,'ones shape:',ones.shape,'stest shape',stest_pred.flatten().shape)
      result = np.where(np.absolute((data[:,-1]-stest_pred.flatten())) > (0.05*np.absolute(data[:,-1])),ones,zeros)
      data[:,-1]=result
      return data
    
def accuracy_estim(labeled_data):
    index_f = np.where(labeled_data[:,-1]==1)
    index_p = np.where(labeled_data[:,-1]==0)
    failed_gt= labeled_data[index_f[0]]
    passed_gt=labeled_data[index_p[0]]
    print('outlier is:',failed_gt.shape[0])
    return (passed_gt.shape[0]/(passed_gt.shape[0]+failed_gt.shape[0]))

In [5]:
train_data= np.loadtxt("./train_2400.txt", delimiter=" ",skiprows=0, dtype=np.float32)
test_data= np.loadtxt("./test_2400.txt", delimiter=" ",skiprows=0, dtype=np.float32)
print(train_data.shape,test_data.shape)
display(train_data)

(2400, 7) (621, 7)


array([[ 253.24956  ,  561.4994   ,  292.8345   , ...,    3.050196 ,
          49.257195 ,    3.8032868],
       [ 242.95137  ,  853.87775  ,  386.51224  , ...,    3.5592268,
          35.442196 ,    5.431416 ],
       [ 304.8533   ,  472.54318  ,  385.01926  , ...,    3.8708582,
          14.153753 ,    5.1082335],
       ...,
       [ 180.0256   , 1539.7944   ,  199.59033  , ...,    4.066501 ,
          14.5367   ,    9.101738 ],
       [ 130.54503  , 1512.5757   ,  132.32954  , ...,    4.5932894,
          49.209843 ,    2.5677352],
       [ 150.96326  , 1095.6674   ,  209.29689  , ...,    2.6020818,
          27.179028 ,    2.7654684]], dtype=float32)

# Random forest

In [30]:
from sklearn.ensemble import RandomForestRegressor
train_X, train_y = train_data[:,:-1],train_data[:,-1]
test_X, test_y = test_data[:,:-1],test_data[:,-1]
print(train_X, train_y)
regr = RandomForestRegressor(random_state=0)

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
print('pred_y :',pred_y.shape,'test_y:',test_y.shape)

copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

copied_test_data=np.copy(test_data)
calc_residual(test_y,pred_y)
calc_deviation(test_y,pred_y)

[[ 253.24956    561.4994     292.8345     107.47464      3.050196
    49.257195 ]
 [ 242.95137    853.87775    386.51224    141.02594      3.5592268
    35.442196 ]
 [ 304.8533     472.54318    385.01926    167.81926      3.8708582
    14.153753 ]
 ...
 [ 180.0256    1539.7944     199.59033    174.2642       4.066501
    14.5367   ]
 [ 130.54503   1512.5757     132.32954     55.579464     4.5932894
    49.209843 ]
 [ 150.96326   1095.6674     209.29689     76.64913      2.6020818
    27.179028 ]] [3.8032868 5.431416  5.1082335 ... 9.101738  2.5677352 2.7654684]
pred_y : (621,) test_y: (621,)
outlier is: 135
accuracy estimation: 0.782608695652174
residual is: (621,) sum residual is: 0.042342332961201164
variance is: 0.7027151983782651
std is: 0.8382810974716447


0.8382810974716447

# XGBoost

In [31]:
from sklearn import ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

regr = ensemble.GradientBoostingRegressor()

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
print('pred_y :',pred_y,'test_y:',test_y)
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))
calc_residual(test_y,pred_y)
calc_deviation(test_y,pred_y)

pred_y : [10.75479619  3.25989114  4.6469407   4.2440031   7.20124235  3.8097412
  2.99945305  6.40742452  4.9358584   4.15210147  3.96141355  7.06488511
  2.82142324  2.24259318  6.31301707  8.53993227  3.73433161  6.60561165
  4.96717849  7.41519477  6.91383869  5.27923603  6.97664019  4.83859655
  4.26361302  7.34565939  5.70973456  8.56382612  5.02570399  4.18588819
  2.82652407  5.18743395  7.46550174  2.14538955  5.25352644  6.20303504
  7.54489924  6.0525072   3.20158226  3.29019678  5.40322331  4.3020322
  7.47534326  9.00728329  4.01912706  4.94979235  7.17975736  5.26905077
  4.85287989  4.90630121  6.78133535  3.71275329  7.17903128  8.9711024
  4.48244772  4.7515431   6.65274966  4.16902626  7.40589372  4.31608509
  4.90520125  2.47752018  4.22265008  7.34531215  5.01200472  4.03441798
  2.98003974  7.73791791  3.62550469  7.8408509   7.26586277  4.8958521
  2.76001979  6.34657261  6.93615731  7.18735245  6.93420858  3.29851468
  2.04315598  7.31049386  2.73549757  3.674471

0.8257288488961715

In [43]:
from sklearn import ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error

regr = ensemble.GradientBoostingRegressor(learning_rate=0.3,n_estimators=50)

regr.fit(train_X, train_y)
pred_y=regr.predict(test_X)
#print('pred_y :',pred_y,'test_y:',test_y)
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

outlier is: 184
accuracy estimation: 0.7037037037037037


# GP regressor

In [53]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,RBF
print('Size of train X:',train_X.shape,'test_X:',test_X.shape)
kernel = RBF(10)
gpr = GaussianProcessRegressor(kernel=kernel).fit(train_X, train_y)

pred_y=gpr.predict(test_X, return_std=False)
print('pred y shape is:',pred_y.shape,'pred_y is:',pred_y)

Size of train X: (2400, 6) test_X: (621, 6)
pred y shape is: (621,) pred_y is: [8.09184144e-04 3.03821279e-04 6.55934038e-04 1.01611795e-02
 1.13949617e-03 3.95793829e-05 3.02850681e-01 2.31887216e-03
 5.79270897e-02 1.54148546e-02 1.48517238e-01 5.95602726e-03
 7.39528043e-02 2.33331966e-01 2.37190525e-12 2.24946967e-06
 5.73812663e-02 1.11710113e-01 3.43149443e-04 3.31405009e-06
 1.04999442e+00 4.93771845e-04 2.15572571e-01 4.40532809e-01
 2.41799386e-03 6.41551294e-05 3.11943467e-08 1.14678513e-01
 8.66307042e-04 5.67318575e-03 1.21561164e-04 3.37772378e-02
 2.40978072e-02 2.13281545e-02 2.91780202e-03 7.82731476e-02
 1.22037092e-01 4.02237001e-02 8.87817418e-01 5.83143925e-02
 2.63696329e-02 7.84960352e-01 1.38005811e-01 9.80198473e-07
 2.70896309e-01 1.78104713e-02 7.69381296e-03 1.22277033e-02
 2.28426463e-01 3.32175480e-01 5.23783466e-01 5.06664091e-02
 4.88091121e-03 1.46235126e+00 1.78375620e-01 4.88709824e-02
 3.03334523e-02 1.81245258e-04 2.30735429e-03 1.89908575e-03
 3.455

In [49]:
copied_test_data=np.copy(test_data)
labeled_data= label_data(copied_test_data,pred_y)
print('accuracy estimation:',accuracy_estim(labeled_data))

outlier is: 621
accuracy estimation: 0.0


In [50]:
print('pred_y :',pred_y,'test_y:',test_y)

pred_y : [8.09184144e-04 3.03821279e-04 6.55934038e-04 1.01611795e-02
 1.13949617e-03 3.95793829e-05 3.02850681e-01 2.31887216e-03
 5.79270897e-02 1.54148546e-02 1.48517238e-01 5.95602726e-03
 7.39528043e-02 2.33331966e-01 2.37190525e-12 2.24946967e-06
 5.73812663e-02 1.11710113e-01 3.43149443e-04 3.31405009e-06
 1.04999442e+00 4.93771845e-04 2.15572571e-01 4.40532809e-01
 2.41799386e-03 6.41551294e-05 3.11943467e-08 1.14678513e-01
 8.66307042e-04 5.67318575e-03 1.21561164e-04 3.37772378e-02
 2.40978072e-02 2.13281545e-02 2.91780202e-03 7.82731476e-02
 1.22037092e-01 4.02237001e-02 8.87817418e-01 5.83143926e-02
 2.63696329e-02 7.84960352e-01 1.38005811e-01 9.80198473e-07
 2.70896309e-01 1.78104713e-02 7.69381296e-03 1.22277033e-02
 2.28426463e-01 3.32175480e-01 5.23783466e-01 5.06664091e-02
 4.88091121e-03 1.46235126e+00 1.78375620e-01 4.88709824e-02
 3.03334523e-02 1.81245258e-04 2.30735429e-03 1.89908575e-03
 3.45526485e-01 8.21813046e-01 9.39256294e-03 2.58870658e-02
 1.04324702e-01